In [ ]:
import os
from langchain_community.utilities import SQLDatabase

# OpenAI API 키 설정 (이 예제에서는 사용되지 않음)
api_key = os.getenv("OPENAI_API_KEY")

# SQLite 데이터베이스 경로
db_path = "/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db"

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 데이터베이스 방언 출력 (예: sqlite)
print(db.dialect)

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

# SQL 쿼리 실행 및 결과 출력
result = db.run("SELECT * FROM Artist LIMIT 10;")
print(result)

In [ ]:
import pprint
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate


from langchain.globals import set_verbose
from langchain.globals import set_debug
set_verbose(True)
set_debug(True)
# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 데이터베이스 쿼리 도구 설정
tools = [QuerySQLDataBaseTool(db=db, description="사용자의 질문이 데이터베이스와 연관된 경우에만 사용하세요.")]
tool_names = [tool.name for tool in tools]

# PromptTemplate 설정
answer_prompt = PromptTemplate.from_template(
    """ 
사용자 질문에 친절하게 답변해 주세요.
만약, 데이터베이스와 관련된 질문인 경우에는 {tool_names}를 사용하여 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.

질문: {question}
SQL 쿼리: {query}
SQL 결과: {result}
답변: 
"""
)


# 에이전트 생성
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    tools=tools,  # 데이터베이스 쿼리용 도구 추가
    agent_type="openai-tools", 
    verbose=True
)

# 사용자 질문 설정
user_question = "대한민국의 수도는?"

formatted_prompt = answer_prompt.format(
    question=user_question,
    tools=tools,
    tool_names=', '.join(tool_names),
    query="{query}",  # Placeholder for query
    result="{result}"  # Placeholder for result
)

response = agent_executor.invoke({"input": formatted_prompt})

print("Agent 답변 :")
pprint.pprint(response)


In [17]:
import logging
import pprint
import json
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain.globals import set_verbose, set_debug
from collections import defaultdict
from datetime import datetime

# Custom logging handler that stores logs in a dictionary and writes to a JSON file
class CustomJSONLoggingHandler(logging.Handler):
    def __init__(self, filename):
        super().__init__()
        self.filename = filename
        self.logs = defaultdict(list)

    def emit(self, record):
        log_entry = self.format(record)
        timestamp = datetime.fromtimestamp(record.created).strftime('%Y-%m-%d %H:%M:%S')
        self.logs[timestamp].append(log_entry)
    
    def save_to_json(self):
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(self.logs, f, ensure_ascii=False, indent=4)

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create a custom handler and formatter
custom_handler = CustomJSONLoggingHandler('debug_logs.json')
formatter = logging.Formatter('%(asctime)s - %(message)s')
custom_handler.setFormatter(formatter)
logger.addHandler(custom_handler)

# Configure LangChain to use logging for debug and verbose output
set_verbose(True)
set_debug(True)

# OpenAI LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 데이터베이스 쿼리 도구 설정
tools = [QuerySQLDataBaseTool(db=db, description="사용자의 질문이 데이터베이스와 연관된 경우에만 사용하세요.")]
tool_names = [tool.name for tool in tools]

# PromptTemplate 설정
answer_prompt = PromptTemplate.from_template(
    """ 
사용자 질문에 친절하게 답변해 주세요.
만약, 데이터베이스와 관련된 질문인 경우에는 {tool_names}를 사용하여 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.

질문: {question}
SQL 쿼리: {query}
SQL 결과: {result}
답변: 
"""
)

# 에이전트 생성
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    tools=tools,  # 데이터베이스 쿼리용 도구 추가
    agent_type="openai-tools", 
    verbose=True
)

# 사용자 질문 설정
user_question = "대한민국의 수도는?"

formatted_prompt = answer_prompt.format(
    question=user_question,
    tools=tools,
    tool_names=', '.join(tool_names),
    query="{query}",  # Placeholder for query
    result="{result}"  # Placeholder for result
)

response = agent_executor.invoke({"input": formatted_prompt})

print("Agent 답변 :")
pprint.pprint(response)

# Save the stored debug information to JSON file
custom_handler.save_to_json()

# Print a message to indicate that logs have been saved
print("Debug logs have been saved to debug_logs.json")


[chain/start] [chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": " \n사용자 질문에 친절하게 답변해 주세요.\n만약, 데이터베이스와 관련된 질문인 경우에는 sql_db_query를 사용하여 사용자 질문, 해당하는 SQL 쿼리, 그리고 SQL 결과를 바탕으로 사용자 질문에 한국어로 답변해 주세요.\n\n질문: 대한민국의 수도는?\nSQL 쿼리: {query}\nSQL 결과: {result}\n답변: \n"
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:SQL Agent